### This notebook handles the transformation of Gazebo experimental data to  a common format to the UE4 data.

The setup used made Gazebo return all the observations in separate .csv files, which is not a clean solution, therefore processing of the .csv files had to be done to combine them in a single .csv file, rename the observation labels and order the observations so that the final result is consistent with the observations from UE4.

In [1]:
import pandas as pd
from glob import glob

In [2]:
def transform_data(source_folder, destination_file, model_name, hasAcceleration=True, hasForce=True):
    # This function takes a folder containing Gazebo results in several .csv files and merges them into 1 .csv file that has 
    # consistent observation labels and order to the UE4 observations.
    stock_files = sorted(glob(source_folder + '*.csv'))
    df = pd.concat((pd.read_csv(file) for file in stock_files), axis=1, ignore_index = False)
    # Remove duplicate columns, initially used to eliminate duplicate sim_time columns.
    df = df.loc[:,~df.columns.duplicated()]
    
    df = df.rename(columns={"sim_time": "Time", model_name + "world_angular_acceleration/x": "X Angular Acceleration", 
                           model_name + "world_angular_velocity/x":"X Angular Velocity", model_name + "world_angular_velocity/y":"Y Angular Velocity",
                           model_name + "world_angular_velocity/z":"Z Angular Velocity", model_name + "world_linear_acceleration/x":"X Linear Acceleration", 
                           model_name + "world_linear_velocity/x":"X Velocity", model_name + "world_linear_velocity/y":"Y Velocity", model_name + "world_linear_velocity/z":"Z Velocity",
                           model_name + "world_pose/orientation/pitch":"Pitch", model_name + "world_pose/orientation/roll":"Roll", model_name + "world_pose/orientation/yaw":"Yaw",
                           model_name + "world_pose/position/x":"X Position", model_name + "world_pose/position/y":"Y Position", model_name + "world_pose/position/z":"Z Position",})
    
    cols = ['Time', 'X Velocity', 'Y Velocity', 'Z Velocity', 'X Position', 'Y Position', 'Z Position', 'Roll', 'Yaw', 'Pitch',
            'X Angular Velocity', 'Y Angular Velocity', 'Z Angular Velocity']
    
    if hasAcceleration == True:
        df = df.rename(columns={model_name + "world_angular_acceleration/y": "Y Angular Acceleration", model_name + "world_angular_acceleration/z": "Z Angular Acceleration",
                                model_name + "world_linear_acceleration/y":"Y Linear Acceleration", model_name + "world_linear_acceleration/z":"Z Linear Acceleration"})
        accelerations = ['X Angular Acceleration', 'Y Angular Acceleration','Z Angular Acceleration', 'X Linear Acceleration', 'Y Linear Acceleration', 'Z Linear Acceleration']
        cols = cols + accelerations
        
    if hasForce == True:
        df = df.rename(columns = {" applied_force_x":"X Force Applied", " applied_force_y":"Y Force Applied", " applied_force_z":"Z Force Applied"})
        # Apply a specific ordering to the columns to be consistent with UE4 observations.
        cols = ['Time', 'X Velocity', 'Y Velocity', 'Z Velocity', 'X Position', 'Y Position', 'Z Position', 'Roll', 'Yaw', 'Pitch',
                'X Angular Velocity', 'Y Angular Velocity', 'Z Angular Velocity', 'X Force Applied', 'Y Force Applied', 'Z Force Applied', 
                'X Angular Acceleration', 'Y Angular Acceleration', 'Z Angular Acceleration', 'X Linear Acceleration', 'Y Linear Acceleration', 'Z Linear Acceleration']

    df = df[cols]

    # Save the formatted dataframe to a .csv file
    df.to_csv(destination_file, index=False)

In [6]:
# Transform the data for the gravity drop experiments
gravity_drop_test_data = 'D:\\University\\Final Year Project\\Final Results\\Gravity Drop Test\\Gazebo Results\\'
transform_data(gravity_drop_test_data, gravity_drop_test_data + "Gravity_Drop_Test_Observations.csv", ' unit_box_Custom?', True, False)

In [3]:
# Transform the data for the gravity drop experiments
# 50 Meters
folder = 'D:\\University\\Final Year Project\\GitHub\\Preliminary Work\\Results\\Gravity Drop Test\\50 Meters\\Gazebo Results\\'
transform_data(folder, folder + "Gravity_Drop_Test_Observations.csv", ' unit_box_Custom?', True, False)

# 75 Meters
folder = 'D:\\University\\Final Year Project\\GitHub\\Preliminary Work\\Results\\Gravity Drop Test\\75 Meters\\Gazebo Results\\'
transform_data(folder, folder + "Gravity_Drop_Test_Observations.csv", ' unit_box_Custom?', True, False)

# 100 Meters
folder = 'D:\\University\\Final Year Project\\GitHub\\Preliminary Work\\Results\\Gravity Drop Test\\100 Meters\\Gazebo Results\\'
transform_data(folder, folder + "Gravity_Drop_Test_Observations.csv", ' unit_box_Custom?', True, False)

In [13]:
# Transform the data for the constant forces 
forces_used = [1, 5, 10, 20]
n = len(forces_used)
for ix in range(0, n):
    gazebo_data_folder_path = 'D:\\University\\Final Year Project\\Final Results\\Constant Force\\Gazebo Quick Solver Results\\' + str(forces_used[ix]) + 'N\\'
    transform_data(gazebo_data_folder_path, gazebo_data_folder_path + str(forces_used[ix]) + "N.csv", ' box?', True, True, )    

In [4]:
# Transform the data for impulse experiments
impulses_used = [1000, 10000]
n = len(impulses_used)
# No friction case
for ix in range(0, n):
    gazebo_data_folder_path = 'D:\\University\\Final Year Project\\GitHub\\Preliminary Work\\Results\\Impulse\\Neglecting_Friction\\Gazebo Results\\' + str(impulses_used[ix]) + 'N\\'
    transform_data(gazebo_data_folder_path, gazebo_data_folder_path + str(impulses_used[ix]) + "N.csv", ' unit_box_Custom?', False, False)    
    
# Friction = 1
for ix in range(0, n):
    gazebo_data_folder_path = 'D:\\University\\Final Year Project\\GitHub\\Preliminary Work\\Results\\Impulse\\Including_Friction\\Gazebo Results\\' + str(impulses_used[ix]) + 'N\\'
    transform_data(gazebo_data_folder_path, gazebo_data_folder_path + str(impulses_used[ix]) + "N.csv", ' unit_box_Custom?', False, False)    

In [23]:
# Transform the data for force pattern experiments
for crt_pattern in range(1, 5):
    gazebo_data_folder_path = 'D:\\University\\Final Year Project\\Final Results\\Same tick duration approach\\Gazebo World Solver Results\\Pattern_' + str(crt_pattern) + '\\'
    
    transform_data(gazebo_data_folder_path, gazebo_data_folder_path + "Pattern_" + str(crt_pattern) + '.csv', ' box?')